<a href="https://colab.research.google.com/github/KIM-Jaegyeong/SBA2019_SeoulIT/blob/master/poject_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 불러오기

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import seaborn as sns
import scipy.stats as stats
sns.set(style='darkgrid')
warnings.filterwarnings('ignore')

In [0]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 한글폰트 사용시 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [0]:
# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

# 데이터 불러오기

In [0]:
train = pd.read_csv("train.csv", sep=',', parse_dates=['Date'],
                    dtype={'StateHoliday': str, 'SchoolHoliday':str})

test = pd.read_csv("test.csv", sep=",", index_col = 'Id', parse_dates=['Date'],
                  dtype={'StateHoliday': str, 'SchoolHoliday':str})

store = pd.read_csv("store.csv", sep=",", dtype={'StoreType': str,
                                                      'Assortment': str,
                                                      'PromoInterval': str})

#컬럼 소개

**Store**

Store : 가게 고유 id

StoreType : 가게 타입 a,b,c,d

Assortment: 분류 레벨 (a = basic, b = 
extra, c = extended )

CompetitionDistance: 경쟁 업체와의 거리
(meters)

CompetitionOpenSinceMonth : 경쟁업체가 open한 월

CompetitionOpenSinceYear : 경쟁업체가 
open한 년도

Promo2 : 0 = store is not participating, 1 
= store is participating

Promo2SinceWeek : 프로모션 참여 시작 주

Promo2SinceYear : 프로모션 참여 시작 달

PromoInterval : 프로모션 하는 달

**Test**

Id: id

Store : 가게 고유 id

dayofweek: the date in a week: 1¬-7 요일?

date : 날짜

open : closed =0 , open =1

promo : 0 = no promo, 1 = promo

StateHoliday : a = public holiday, b = 
Easter holiday, c = Christmas, 0 = None

SchoolHoliday : : 1 = school holiday, 0 = not school holiday

**Train**

Store: 가게 고유 id

Dayofweek : the date in a week: 1¬-7 요일?


Date : 날짜

Sales : the turnover for any given day: 
integer number (This is what to be predict)
 판매수?

Customers : 지정 날짜의 고객수 ( test data에 없음)

open : closed =0 , open =1

promo : 0 = no promo, 1 = promo

StateHoliday : a = public holiday, b = Easter holiday, c = Christmas, 0 = None

SchoolHoliday : : 1 = school holiday, 0 = not school holiday

# 데이터 탐색

**1) Train Dataset**

In [0]:
print(train.head())

   Store  DayOfWeek       Date    Sales  ...  Open  Promo  StateHoliday SchoolHoliday
0      1          5 2015-07-31   5263.0  ...   1.0    1.0             0             1
1      2          5 2015-07-31   6064.0  ...   1.0    1.0             0             1
2      3          5 2015-07-31   8314.0  ...   1.0    1.0             0             1
3      4          5 2015-07-31  13995.0  ...   1.0    1.0             0             1
4      5          5 2015-07-31   4822.0  ...   1.0    1.0             0             1

[5 rows x 9 columns]


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420077 entries, 0 to 420076
Data columns (total 9 columns):
Store            420077 non-null int64
DayOfWeek        420077 non-null int64
Date             420077 non-null datetime64[ns]
Sales            420076 non-null float64
Customers        420076 non-null float64
Open             420076 non-null float64
Promo            420076 non-null float64
StateHoliday     420076 non-null object
SchoolHoliday    420076 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 28.8+ MB


- StateHoliday, StateHoliday 범주형 -> 실수형
- 행 1017208개, 컬럼 9개
- - 범주형 변수 : DayOfWeek, Open,Promo (StateHoliday, SchoolHoliday object->변환하기)

# 컬럼 정리

In [0]:
# date - 년,월 분류
train['Year'] = pd.DatetimeIndex(train['Date']).year
train['Month'] = pd.DatetimeIndex(train['Date']).month

In [0]:
train = train[['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Customers', 'Open',
               'Promo', 'StateHoliday', 'SchoolHoliday', 'Sales']]
list(train.columns.values)

['Store',
 'DayOfWeek',
 'Date',
 'Year',
 'Month',
 'Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'Sales']

In [0]:
train['StateHoliday'].unique()

array(['0', 'a', 'b', 'c', nan], dtype=object)

In [0]:
# train.loc[train['StateHoliday'] == '0', 'StateHoliday'] = 0
# train.loc[train['StateHoliday'] == 'a', 'StateHoliday'] = 1
# train.loc[train['StateHoliday'] == 'b', 'StateHoliday'] = 2
# train.loc[train['StateHoliday'] == 'c', 'StateHoliday'] = 3
# train['StateHoliday'] = train['StateHoliday'].astype(int, copy=False)

In [0]:
# int로 만드는 function
def factor_to_integer(df, colname, start_value=0):
    while df[colname].dtype == object:
        myval = start_value # factor starts at "start_value".
        for sval in df[colname].unique():
            df.loc[df[colname] == sval, colname] = myval
            myval += 1
        df[colname] = df[colname].astype(int, copy=False)
    print('levels :', df[colname].unique(), '; data type :', df[colname].dtype)

In [0]:
# factor_to_integer(train, 'SchoolHoliday')

In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420077 entries, 0 to 420076
Data columns (total 11 columns):
Store            420077 non-null int64
DayOfWeek        420077 non-null int64
Date             420077 non-null datetime64[ns]
Year             420077 non-null int64
Month            420077 non-null int64
Customers        420076 non-null float64
Open             420076 non-null float64
Promo            420076 non-null float64
StateHoliday     420076 non-null object
SchoolHoliday    420076 non-null object
Sales            420076 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 35.3+ MB


In [0]:
print(train.describe())

               Store      DayOfWeek  ...          Promo          Sales
count  420077.000000  420077.000000  ...  420076.000000  420076.000000
mean      558.674695       4.002492  ...       0.394181    5935.773715
std       321.979296       1.999417  ...       0.488675    3891.595052
min         1.000000       1.000000  ...       0.000000       0.000000
25%       281.000000       2.000000  ...       0.000000    3890.000000
50%       558.000000       4.000000  ...       0.000000    5921.000000
75%       838.000000       6.000000  ...       1.000000    8051.000000
max      1115.000000       7.000000  ...       1.000000   41551.000000

[8 rows x 8 columns]


In [0]:
#null 확인
from collections import Counter
x = {colname : train[colname].isnull().sum() for colname in train.columns}
Counter(x).most_common()

[('Customers', 1),
 ('Open', 1),
 ('Promo', 1),
 ('StateHoliday', 1),
 ('SchoolHoliday', 1),
 ('Sales', 1),
 ('Store', 0),
 ('DayOfWeek', 0),
 ('Date', 0),
 ('Year', 0),
 ('Month', 0)]
















**2) Test Dataset**

In [0]:
print(test.head())

    Store  DayOfWeek       Date  Open  Promo StateHoliday SchoolHoliday
Id                                                                     
1       1          4 2015-09-17   1.0      1            0             0
2       3          4 2015-09-17   1.0      1            0             0
3       7          4 2015-09-17   1.0      1            0             0
4       8          4 2015-09-17   1.0      1            0             0
5       9          4 2015-09-17   1.0      1            0             0


In [0]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 1 to 41088
Data columns (total 7 columns):
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Date             41088 non-null datetime64[ns]
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 2.5+ MB


- StateHoliday, StateHoliday 범주형 -> 실수형
- open float?
- 행 41088개, 컬럼 7개

# 컬럼 정리

In [0]:
test['Year'] = pd.DatetimeIndex(test['Date']).year
test['Month'] = pd.DatetimeIndex(test['Date']).month

In [0]:
test = test[['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Open',
             'Promo', 'StateHoliday', 'SchoolHoliday']]
list(test.columns.values)

['Store',
 'DayOfWeek',
 'Date',
 'Year',
 'Month',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday']

In [0]:
#null 확인
x = {colname : test[colname].isnull().sum() for colname in test.columns}
Counter(x).most_common()

[('Open', 11),
 ('Store', 0),
 ('DayOfWeek', 0),
 ('Date', 0),
 ('Year', 0),
 ('Month', 0),
 ('Promo', 0),
 ('StateHoliday', 0),
 ('SchoolHoliday', 0)]

In [0]:
# store 622가 NaN값 
print(test.ix[np.isnan(test['Open'])])

       Store  DayOfWeek       Date  ...  Promo  StateHoliday  SchoolHoliday
Id                                  ...                                    
480      622          4 2015-09-17  ...      1             0              0
1336     622          3 2015-09-16  ...      1             0              0
2192     622          2 2015-09-15  ...      1             0              0
3048     622          1 2015-09-14  ...      1             0              0
4760     622          6 2015-09-12  ...      0             0              0
5616     622          5 2015-09-11  ...      0             0              0
6472     622          4 2015-09-10  ...      0             0              0
7328     622          3 2015-09-09  ...      0             0              0
8184     622          2 2015-09-08  ...      0             0              0
9040     622          1 2015-09-07  ...      0             0              0
10752    622          6 2015-09-05  ...      0             0              0

[11 rows x 

In [0]:
print(train.ix[train['Store'] == 622].head())

      Store  DayOfWeek       Date  ...  StateHoliday  SchoolHoliday   Sales
621     622          5 2015-07-31  ...             0              0  6306.0
1736    622          4 2015-07-30  ...             0              0  5412.0
2851    622          3 2015-07-29  ...             0              0  5326.0
3966    622          2 2015-07-28  ...             0              0  4966.0
5081    622          1 2015-07-27  ...             0              0  5413.0

[5 rows x 11 columns]


In [0]:
test.ix[np.isnan(test['Open']), 'Open'] = 1

In [0]:
from collections import Counter
x = {colname : test[colname].isnull().sum() for colname in test.columns}
Counter(x).most_common()

[('Store', 0),
 ('DayOfWeek', 0),
 ('Date', 0),
 ('Year', 0),
 ('Month', 0),
 ('Open', 0),
 ('Promo', 0),
 ('StateHoliday', 0),
 ('SchoolHoliday', 0)]

In [0]:
test['StateHoliday'].unique()

array(['0', 'a'], dtype=object)

In [0]:

test['SchoolHoliday'].unique()

array(['0', '1'], dtype=object)

In [0]:




factor_to_integer(test, 'StateHoliday')
factor_to_integer(test, 'SchoolHoliday')
test.dtypes

levels : [0 1] ; data type : int64
levels : [0 1] ; data type : int64


Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Year                      int64
Month                     int64
Open                    float64
Promo                     int64
StateHoliday              int64
SchoolHoliday             int64
dtype: object

In [0]:
# holiday 맞추기 (StateHoliday : a = public holiday, b = Easter holiday, c = Christmas, 0 = None)
# train = train.loc[train['StateHoliday'] < 2]

**3) Store Dataset**

In [0]:



print(test.head())

    Store  DayOfWeek       Date  Year  ...  Open  Promo  StateHoliday  SchoolHoliday
Id                                     ...                                          
1       1          4 2015-09-17  2015  ...   1.0      1             0              0
2       3          4 2015-09-17  2015  ...   1.0      1             0              0
3       7          4 2015-09-17  2015  ...   1.0      1             0              0
4       8          4 2015-09-17  2015  ...   1.0      1             0              0
5       9          4 2015-09-17  2015  ...   1.0      1             0              0

[5 rows x 9 columns]


In [0]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1112 non-null float64
CompetitionOpenSinceMonth    761 non-null float64
CompetitionOpenSinceYear     761 non-null float64
Promo2                       1115 non-null int64
Promo2SinceWeek              571 non-null float64
Promo2SinceYear              571 non-null float64
PromoInterval                571 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [0]:
x = {colname : store[colname].isnull().sum() for colname in store.columns}
Counter(x).most_common()

[('Promo2SinceWeek', 544),
 ('Promo2SinceYear', 544),
 ('PromoInterval', 544),
 ('CompetitionOpenSinceMonth', 354),
 ('CompetitionOpenSinceYear', 354),
 ('CompetitionDistance', 3),
 ('Store', 0),
 ('StoreType', 0),
 ('Assortment', 0),
 ('Promo2', 0)]

In [0]:
store['PromoInterval'].unique()

array([nan, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec'],
      dtype=object)

In [0]:





store.ix[store['Promo2'] == 0, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']] = 0

In [0]:
store.ix[store['Promo2'] != 0, 'Promo2SinceWeek'] = store['Promo2SinceWeek'].max() - store.ix[store['Promo2'] != 0, 'Promo2SinceWeek']

In [0]:
store.ix[store['Promo2'] != 0, 'Promo2SinceYear'] = store['Promo2SinceYear'].max() - store.ix[store['Promo2'] != 0, 'Promo2SinceYear']

In [0]:
factor_to_integer(store, 'PromoInterval', start_value=0)

levels : [0 1 2 3] ; data type : int64


In [0]:
factor_to_integer(store, 'StoreType')
factor_to_integer(store, 'Assortment')
store.dtypes

levels : [0 1 2 3] ; data type : int64
levels : [0 1 2] ; data type : int64


Store                          int64
StoreType                      int64
Assortment                     int64
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                  int64
dtype: object

In [0]:

x = {colname : store[colname].isnull().sum() for colname in store.columns}
Counter(x).most_common()

[('CompetitionOpenSinceMonth', 354),
 ('CompetitionOpenSinceYear', 354),
 ('CompetitionDistance', 3),
 ('Store', 0),
 ('StoreType', 0),
 ('Assortment', 0),
 ('Promo2', 0),
 ('Promo2SinceWeek', 0),
 ('Promo2SinceYear', 0),
 ('PromoInterval', 0)]

In [0]:
from sklearn.preprocessing import Imputer
imputer = Imputer().fit(store)
store_imputed = imputer.transform(store)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
store2 = pd.DataFrame(store_imputed, columns=store.columns.values)

In [0]:
x = {colname : store2[colname].isnull().sum() for colname in store2.columns}
Counter(x).most_common()

[('Store', 0),
 ('StoreType', 0),
 ('Assortment', 0),
 ('CompetitionDistance', 0),
 ('CompetitionOpenSinceMonth', 0),
 ('CompetitionOpenSinceYear', 0),
 ('Promo2', 0),
 ('Promo2SinceWeek', 0),
 ('Promo2SinceYear', 0),
 ('PromoInterval', 0)]

In [0]:
store2['CompetitionOpenSinceMonth'] = store2['CompetitionOpenSinceMonth'].max() - store2['CompetitionOpenSinceMonth']

In [0]:
store2['CompetitionOpenSinceYear'] = store2['CompetitionOpenSinceYear'].max() - store2['CompetitionOpenSinceYear']

In [0]:
len(store2['Store']) - sum(store2['Store'].isin(train['Store']))

0

In [0]:
StoreStore = pd.Series(store2['Store']); StoreTrain = pd.Series(train['Store'])

sum(StoreTrain.isin(StoreStore) == False)

0

In [0]:
train_store = pd.merge(train, store2, how = 'left', on='Store')

In [0]:
print(train_store.head())

   Store  DayOfWeek       Date  ...  Promo2SinceWeek  Promo2SinceYear  PromoInterval
0      1          5 2015-07-31  ...              0.0              0.0            0.0
1      2          5 2015-07-31  ...             37.0              5.0            1.0
2      3          5 2015-07-31  ...             36.0              4.0            1.0
3      4          5 2015-07-31  ...              0.0              0.0            0.0
4      5          5 2015-07-31  ...              0.0              0.0            0.0

[5 rows x 20 columns]


In [0]:
test_store = test.reset_index().merge(store2, how = 'left', on='Store').set_index('Id')

In [0]:

print(test_store.head())

    Store  DayOfWeek  ... Promo2SinceYear  PromoInterval
Id                    ...                               
1       1          4  ...             0.0            0.0
2       3          4  ...             4.0            1.0
3       7          4  ...             0.0            0.0
4       8          4  ...             0.0            0.0
5       9          4  ...             0.0            0.0

[5 rows x 18 columns]


In [0]:
test_store.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Year                         0
Month                        0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [0]:
train_store.columns

Index(['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Sales', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [0]:
test_store.columns

Index(['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [0]:
drop_columns = ["상권업종중분류코드", "상권업종소분류코드", 
                "건물관리번호", "표준산업분류코드", "표준산업분류명", 
                "지번부번지", "층정보", "건물명", "호정보", 
                "지점명", "건물부번지", "동정보"]

# 모델링


In [0]:
train_store.columns.values

array(['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Sales', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'], dtype=object)

In [0]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

In [0]:
# target , feature
# feature_cols =[ 'Store', 
#     'CompetitionDistance', 
#     'Promo', 
#     'Promo2', 
#     'SchoolHoliday',
#     'StateHoliday',
#     'StoreType',
#     'Assortment',
#     'DayOfWeek',
#     'Month',
#     'Year']
# X = train_model[feature_cols] # Feature
# y = train_model['Sales']  # Target variable

In [0]:
# X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=777 )

In [0]:
# X = train_model.drop(['Sales', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth','Promo2SinceWeek','Promo2SinceYear'], axis=1)
# Y = train_model["Sales"]
# X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [0]:
# Logistic Regression
# from sklearn.linear_model import LogisticRegression

# logreg = LogisticRegression()
# logreg.fit(X_train, y_train)
# y_pred = logreg.predict(X_val)
# acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
# print(acc_logreg)

In [0]:
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.model_selection import cross_validate
# from sklearn.utils import shuffle
# from sklearn.metrics import mean_squared_error, explained_variance_score

In [0]:
# X, y = shuffle(X,y,random_state=10)

In [0]:
# X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=10) # 70% train

In [0]:
# from sklearn.tree import DecisionTreeRegressor
# regressor = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=400, random_state=10)
# regressor.fit(X_train, y_train)

In [0]:
# y_pred = regressor.predict(X_train)
# mse = mean_squared_error(y_test,y_pred)
# evs = explained_variance_score(y_test,y_pred)


**DecisionTree**

In [0]:
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier

In [0]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) # 70% train

In [0]:
# print(X_test.columns)
# print(X_train.columns)
# print(y_train.shape)

In [0]:
# # 의사결정 트리 모델 생성
# clf = DecisionTreeClassifier()
# # 학습
# clf = clf.fit(X_train,y_train)
# # 예측
# y_pred = clf.predict(X_test)

In [0]:
# # 모델평가
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
# from sklearn.ensemble import AdaBoostRegressor

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

# model = AdaBoostRegressor(base_estimator=Lasso())
# model.fit(X_train, y_train)

In [0]:
# #모델 검증
# print(model.score(X_train, y_train)) 
# print(model.score(X_test, y_test))